In [2]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.cbook import boxplot_stats

In [3]:
import seaborn as sns
import sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
import glob

In [4]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/DDoSdata.csv


## 1. Data Prep.

In [5]:
df = pd.read_csv('/kaggle/input/DDoSdata.csv', delimiter=',')
df = df.drop(df.columns[[0,1]],axis=1)
df.dataframeName = 'DDoSdata.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are 1927101 rows and 45 columns


In [6]:
df.head()

,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,pkts,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,1.528103e+09,e,1,tcp,1,192.168.100.150,54110,192.168.100.3,80,10,...,1.21662,40,38,1.56093,1.21662,328,308,1,DDoS,HTTP
1,1.528103e+09,e,1,tcp,1,192.168.100.150,54112,192.168.100.3,80,10,...,1.21662,40,38,1.56107,1.21662,328,308,1,DDoS,HTTP
2,1.528103e+09,e,1,tcp,1,192.168.100.150,54114,192.168.100.3,80,8,...,1.21662,40,38,1.24980,1.21662,328,308,1,DDoS,HTTP
3,1.528103e+09,e,1,tcp,1,192.168.100.150,54116,192.168.100.3,80,8,...,1.21662,40,38,1.24986,1.21662,328,308,1,DDoS,HTTP
4,1.528103e+09,e,1,tcp,1,192.168.100.150,54118,192.168.100.3,80,8,...,1.21662,40,38,1.24991,1.21662,328,308,1,DDoS,HTTP


In [7]:
num_df = df
li_cat = df.select_dtypes(['object']).columns.to_list()
le = LabelEncoder()
for col in li_cat:
    num_df[col] = le.fit_transform(num_df[col].astype(str))

### Lasso Regression
This is used for feature selection.

In [8]:
from sklearn.linear_model import Lasso, MultiTaskLasso, MultiTaskElasticNet
def feature_selection(x,y): 
    lasso_selection = Lasso(alpha=0.3,max_iter=20000)
    lasso_coef = lasso_selection.fit(x,y).coef_
    print('Selected features: ',str(len(lasso_coef[lasso_coef!=0.0])))

    # Making a list from selected features
    val = lasso_coef[lasso_coef!=0.0]
    key, = np.where(lasso_coef!=0.0)
    feature_list = x.columns[key]
    val_plt = np.multiply(val,1000)
    feature_list = feature_list.tolist()
    feature_column = key.tolist()
    val = val.tolist()
    
    print(feature_list)
    return x[feature_list]
for col in ['attack','subcategory']:#'attack','subcategory'-multi-label classification, binary classification
    globals()[f"x_{col}"] = feature_selection(num_df.iloc[:,0:42], num_df[col])
    print("Feature selection finished.")

Selected features:  8
['stime', 'sport', 'ltime', 'seq', 'TnBPSrcIP', 'TnBPDstIP', 'TnP_PerProto', 'Pkts_P_State_P_Protocol_P_DestIP']
Feature selection finished.
Selected features:  17
['stime', 'flgs', 'sport', 'pkts', 'bytes', 'seq', 'dur', 'sbytes', 'srate', 'TnBPSrcIP', 'TnBPDstIP', 'TnP_PSrcIP', 'TnP_PDstIP', 'TnP_PerProto', 'AR_P_Proto_P_DstIP', 'Pkts_P_State_P_Protocol_P_DestIP', 'Pkts_P_State_P_Protocol_P_SrcIP']
Feature selection finished.


In [17]:
from sklearn.model_selection import train_test_split
X_binary,y_binary = np.array(x_attack), np.array(num_df.iloc[:,-3:-2])
y_binary = y_binary.reshape(y_binary.shape[0],)
X_binary_train, X_binary_test, y_binary_train, y_binary_test = train_test_split(X_binary, y_binary, test_size=0.3, random_state=42)

In [18]:
X_multi,y_multi = np.array(x_subcategory), np.array(num_df.iloc[:,-1])
X_multi_train, X_multi_test, y_multi_train, y_multi_test = train_test_split(X_multi, y_multi, test_size=0.3, random_state=42)

In [19]:
from sklearn.metrics import precision_recall_curve, confusion_matrix, classification_report

### Build a Neural Network

In [21]:
dim_binary = len(x_attack.columns.to_list())
dim_binary

8

In [37]:
np.random.seed(42) 
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.losses import BinaryCrossentropy
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from keras.optimizers import Adam
from keras.metrics import Precision, Recall, TruePositives as tp, TrueNegatives as tn, FalseNegatives as fn, FalsePositives as fp

Firstly - the binary classification.

In [39]:
def binary_nn(dim):
    model = Sequential()
    model.add(Dense(1024,input_dim=dim,activation='relu'))  
    model.add(Dropout(0.01))
    model.add(Dense(512,activation='relu'))  
    model.add(Dropout(0.01))
    model.add(Dense(256,activation='relu'))  
    model.add(Dropout(0.01))
    model.add(Dense(128,activation='relu'))  
    model.add(Dropout(0.01))
    model.add(Dense(64,activation='relu'))  
    model.add(Dropout(0.01))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss=BinaryCrossentropy(from_logits=False,label_smoothing=0.1),optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy',Recall(thresholds=0.5), Precision(thresholds=0.5), tp(thresholds=0.5), tn(thresholds=0.5), fp(thresholds=0.5), fn(thresholds=0.5)])
    return model

In [40]:
model_binary = binary_nn(dim_binary)
model_binary.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 1024)              9216      
_________________________________________________________________
dropout_16 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_17 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_18 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 128)              

In [41]:
model_binary.fit(X_binary_train, y_binary_train, batch_size=64, epochs=100, validation_data=(X_binary_test, y_binary_test),
                 callbacks=[CSVLogger('/kaggle/working/training_set_analysis_binary.csv',separator=',', append=False),
                           EarlyStopping(monitor='val_accuracy', patience=3, verbose=1, restore_best_weights=False) ])

Train on 1348970 samples, validate on 578131 samples
Epoch 1/100
1348970/1348970 [==============================] - 323s 239us/step - loss: 27045.9215 - accuracy: 0.9935 - recall_1: 0.9938 - precision_1: 0.9998 - true_positives_1: 1340256.0000 - true_negatives_1: 2.0000 - false_positives_1: 334.0000 - false_negatives_1: 8378.0000 - val_loss: 0.1992 - val_accuracy: 0.9998 - val_recall_1: 1.0000 - val_precision_1: 0.9998 - val_true_positives_1: 577990.0000 - val_true_negatives_1: 0.0000e+00 - val_false_positives_1: 141.0000 - val_false_negatives_1: 0.0000e+00
Epoch 2/100
1348970/1348970 [==============================] - 318s 235us/step - loss: 0.8656 - accuracy: 0.9997 - recall_1: 1.0000 - precision_1: 0.9998 - true_positives_1: 1348628.0000 - true_negatives_1: 0.0000e+00 - false_positives_1: 336.0000 - false_negatives_1: 6.0000 - val_loss: 0.1992 - val_accuracy: 0.9998 - val_recall_1: 1.0000 - val_precision_1: 0.9998 - val_true_positives_1: 577990.0000 - val_true_negatives_1: 0.0000e+0

In [42]:
model_binary.evaluate(X_binary_test, y_binary_test)

578131/578131 [==============================] - 34s 59us/step


[0.19916171061745436,
 0.9997560977935791,
 1.0,
 0.9997560977935791,
 577990.0,
 0.0,
 141.0,
 0.0]

Finally, we take a closer look at the training process.

In [52]:
process = pd.read_csv('/kaggle/working/training_set_analysis_binary.csv')
process.head()

,epoch,accuracy,false_negatives_1,false_positives_1,loss,precision_1,recall_1,true_negatives_1,true_positives_1,val_accuracy,val_false_negatives_1,val_false_positives_1,val_loss,val_precision_1,val_recall_1,val_true_negatives_1,val_true_positives_1
0,0,0.993542,8378.0,334.0,27045.921457,0.999751,0.993788,2.0,1340256.0,0.999756,0.0,141.0,0.199170,0.999756,1.0,0.0,577990.0
1,1,0.999746,6.0,336.0,0.865553,0.999751,0.999996,0.0,1348628.0,0.999756,0.0,141.0,0.199162,0.999756,1.0,0.0,577990.0
2,2,0.999751,0.0,336.0,0.199200,0.999751,1.000000,0.0,1348634.0,0.999756,0.0,141.0,0.199163,0.999756,1.0,0.0,577990.0
3,3,0.999751,0.0,336.0,0.199193,0.999751,1.000000,0.0,1348634.0,0.999756,0.0,141.0,0.199162,0.999756,1.0,0.0,577990.0


In [72]:
def for_ploty(df):
    x = df.set_index('epoch').stack()
    x.index.set_names(['epoch', 'value'], inplace=True)
    x.name = 'evaluation'
    x = x.reset_index()
    print(x)
    return x

In [73]:
for_p = for_ploty(process)

    epoch                 value     evaluation
0       0              accuracy       0.993542
1       0     false_negatives_1    8378.000000
2       0     false_positives_1     334.000000
3       0                  loss   27045.921457
4       0           precision_1       0.999751
..    ...                   ...            ...
59      3              val_loss       0.199162
60      3       val_precision_1       0.999756
61      3          val_recall_1       1.000000
62      3  val_true_negatives_1       0.000000
63      3  val_true_positives_1  577990.000000

[64 rows x 3 columns]


#### Accuracy

In [75]:
import plotly.express as px
df = for_p.query("value in ['accuracy','val_accuracy']")
fig = px.line(df, x="epoch", y="evaluation", color="value")
fig.update_traces(textposition="bottom right")
fig.show()

#### loss

In [76]:
df = for_p.query("value in ['loss','val_loss']")
fig = px.line(df, x="epoch", y="evaluation", color="value")
fig.update_traces(textposition="bottom right")
fig.show()